In [7]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

The aim of the following functions is to create an artificial dataset to test multi-layer perceptron with backpropagation algorithm. Here I have made functions to create datasets for binary classification tasks and for regression tasks.

The idea is to generate sets of k points (one for each class) at random and use the distance from these points to determine the class or the regression target. The precise way to integrate the distance information from k points into a single value is to return the product of the distance from the closest point times the distance from the furthest point.

a) out = oracle(x, landmarks) which takes in input a vector x and a data matrix landmarks that contains a number of landmark vectors as rows. The function must consider only the closest and the furthest landmark to x and return the scalar value of the product of these two distances.

b) (data_mtx, y) or (data_mtx, y, landmarks) = make_dataset_regression(size=100, complexity=2, ndim=3, return_landmarks=False) which outputs the data matrix data_mtx and the target vector y. The number of vectors in data_mtx is determined by the parameter size. The number of landmarks is determined by the parameter complexity. The number of dimensions of the landmark vectors and of the data matrix is determined by ndim. If the flag return_landmarks is set to True then the output of the function includes in addition the landmarks data matrix. The target vector y is computed using the function oracle. The vectors returned are sampled uniformly at random in the unit hyper-cube (i.e. the coordinates are between 0 and 1). The landmarks vectors are sampled uniformly at random in the unit hyper-cube.

c) data_mtx, y = make_2d_grid_dataset_regression(size, landmarks) which outputs the data matrix data_mtx and the target vector y. The number of vectors in data_mtx is determined by the parameter size. The target vector y is computed using the function oracle using the landmarks passed in input in the data matrix landmarks. This function assumes that the dimension of the required vectors is 2 and generates a 2D grid of equally spaced vectors to fill the unit square (i.e. the coordinates are between 0 and 1). Note: if the size is 100 then the grid will be 10 x 10.

d) y = oracle_classification(X, pos_landmarks, neg_landmarks) which takes in input a data matrix X and two data matrix pos_landmarks and neg_landmarks. The function will output a target vector containing the values 1 and 0: a vector x is associated to the target 1 when the value of the oracle function for the positive landmarks is less or equal to the value of the oracle function for the negative landmarks, 0 otherwise.

e) (data_mtx, y) or (data_mtx, y, pos_landmarks, neg_landmarks) = make_dataset_classification(size=100, complexity=2, ndim=3, return_landmarks=False) which outputs the data matrix data_mtx and the target vector y. The number of vectors in data_mtx is determined by the parameter size. The number of landmarks is determined by the parameter complexity. The number of dimensions of the landmark vectors and of the data matrix is determined by ndim. If the flag return_landmarks is set to True then the output of the function includes in addition the positive and negative landmarks data matrices. The target vector y is computed using the function oracle_classification. The vectors returned are sampled uniformly at random in the unit hyper-cube (i.e. the coordinates are between 0 and 1). The landmarks vectors are sampled uniformly at random in the unit hyper-cube.

f) data_mtx, y = make_2d_grid_dataset_classification(size, pos_landmarks, neg_landmarks) which outputs the data matrix data_mtx and the target vector y. The number of vectors in data_mtx is determined by the parameter size. The target vector y is computed using the function oracle_classification using the landmarks passed in input in the data matrices pos_landmarks and neg_landmarks. This function assumes that the dimension of the required vectors is 2 and generates a 2D grid of equally spaced vectors to fill the unit square (i.e. the coordinates are between 0 and 1). Note: if the size is 100 then the grid will be 10 x 10.

In [8]:
def oracle(x, landmarks):
    dist=np.zeros(np.shape(landmarks)[0])
    for i in range(np.shape(landmarks)[0]):
        dist[i]=np.linalg.norm(x-landmarks[i])
    return dist[np.argsort(dist)[0]]*dist[np.argsort(dist)[-1]]

def make_dataset_regression(size=100, complexity=2, ndim=3, return_landmarks=False):
    landmarks=np.zeros([complexity,ndim],dtype=float)
    for i in range(complexity):
        for j in range(ndim):
            landmarks[i][j]=np.random.uniform(low=0, high=1)
    data_mtx=np.zeros([size,ndim],dtype=float)
    y=np.zeros(size,dtype=float)
    for i in range(size):
        for j in range(ndim):
            data_mtx[i][j]=np.random.uniform(low=0, high=1)
        y[i]=oracle(data_mtx[i],landmarks)
    if return_landmarks==True:
        return data_mtx, y, landmarks
    else:
        return data_mtx, y
    
def make_2d_grid_dataset_regression(size, landmarks):
    n=int(np.sqrt(size))
    points=np.linspace(0,1,num=n)
    data_mtx=np.array(np.meshgrid(points,points)).T.reshape(-1,2)
    y=np.zeros(n*n,dtype=float)
    for i in range(len(y)):
        y[i]=oracle(data_mtx[i],landmarks)
    return data_mtx,y
    
def oracle_classification(X, pos_landmarks, neg_landmarks):
    y=np.zeros(np.shape(X)[0],dtype=float)
    for i in range(np.shape(X)[0]):
        p1=oracle(X[i],pos_landmarks)
        p2=oracle(X[i],neg_landmarks)
        if p1<=p2:
            y[i]=1
        else:
            y[i]=0
    return y

def make_dataset_classification(size=100, complexity=2, ndim=3, return_landmarks=False):
    pos_landmarks=np.zeros([complexity,ndim],dtype=float)
    neg_landmarks=np.zeros([complexity,ndim],dtype=float)
    for i in range(complexity):
        for j in range(ndim):
            pos_landmarks[i][j]=np.random.uniform(low=0, high=1)
            neg_landmarks[i][j]=np.random.uniform(low=0, high=1)
    data_mtx=np.zeros([size,ndim],dtype=float)
    
    for i in range(size):
        for j in range(ndim):
            data_mtx[i][j]=np.random.uniform(low=0, high=1)
            
    y=oracle_classification(data_mtx, pos_landmarks, neg_landmarks)
    
    if return_landmarks==True:
        return data_mtx, y, pos_landmarks, neg_landmarks
    else:
        return data_mtx, y
    
def make_2d_grid_dataset_classification(size, pos_landmarks, neg_landmarks):
    n=int(np.sqrt(size))
    points=np.linspace(0,1,num=n,endpoint=False)
    data_mtx=np.array(np.meshgrid(points,points)).T.reshape(-1,2)
    y=oracle_classification(data_mtx, pos_landmarks, neg_landmarks)
    
    return data_mtx,y

plot_2d_classification(X_test, y_test, preds, pos_landmarks, neg_landmarks) to display a data matrix X_test, colouring the instances according to their respective class. The data matrices pos_landmarks and neg_landmarks are also received in input to localize the landmarks. The function should plot three plots: the first where the instances in X_test are coloured using the target vector preds, the second using the target vector y_test and the third where the two coloring schemes are overlapped. To obtain the overlap effect consider using the color map gray for y_test with an alpha of 0.3. The last plot should report in the title the accuracy score of the predictions.

plot_2d_regression(X_test, y_test, preds, landmarks) to display a data matrix X_test, colouring the instances according to their respective class. The data matrix landmarksis also received in input to localize the landmarks. The function should plot two plots: the first where the instances in X_test are coloured using the target vector preds, the second using the target vector y_test. The first plot should report in the title the correlation coefficient score of the predictions vs. the values in y_test

In [9]:
def plot_2d_classification(X_test, y_test, preds, pos_landmarks, neg_landmarks):
    fig,ax=plt.subplots(nrows=1,ncols=3,figsize=(10,5))
    x_coords=[]
    y_coords=[]
    for i in range(np.shape(X_test)[0]):
        x_coords.append(X_test[i][0])
        y_coords.append(X_test[i][1])
        

    ax[0].scatter(x_coords,y_coords,c=preds,cmap='bwr',alpha=0.2)
    ax[0].set_title('Prediction')

    for i in range(len(neg_landmarks)):
        ax[0].plot(pos_landmarks[i][0],pos_landmarks[i][1],marker='d',markersize=10,color='blue')
        ax[0].plot(neg_landmarks[i][0],neg_landmarks[i][1],marker='d',markersize=10,color='red')


    ax[1].scatter(x_coords,y_coords,c=y_test,cmap='bwr',alpha=0.2)
    ax[1].set_title('Truth')

    accuracy=np.sum(preds==y_test)/len(preds)
    for i in range(len(neg_landmarks)):
        ax[1].plot(pos_landmarks[i][0],pos_landmarks[i][1],marker='d',markersize=10,color='blue')
        ax[1].plot(neg_landmarks[i][0],neg_landmarks[i][1],marker='d',markersize=10,color='red')

    ax[2].scatter(x_coords,y_coords,c=preds,cmap='bwr',alpha=0.3)
    ax[2].scatter(x_coords,y_coords,c=y_test,cmap='bwr',alpha=0.1)
    ax[2].set_title(f'Comparison Acc:{round(accuracy,3)}')

    for i in range(len(neg_landmarks)):
        ax[2].plot(pos_landmarks[i][0],pos_landmarks[i][1],marker='d',markersize=10,color='blue')
        ax[2].plot(neg_landmarks[i][0],neg_landmarks[i][1],marker='d',markersize=10,color='red')
    
def plot_2d_regression(X_test, y_test, preds, landmarks):
    fig,ax=plt.subplots(nrows=1,ncols=2,figsize=(10,5))
    x_coords=[]
    y_coords=[]

    for i in range(np.shape(X_test)[0]):
        x_coords.append(X_test[i][0])
        y_coords.append(X_test[i][1])

    correlation_mat=sp.stats.pearsonr(y_test,preds)
    ax[0].scatter(x_coords,y_coords,c=preds,cmap='hot_r')
    ax[0].set_title(f'Prediction cc {round(correlation_mat[0],3)}')
    for i in range(len(landmarks)):
        ax[0].plot(landmarks[i][0],landmarks[i][1],marker='d',markersize=10,color='blue')
        

    ax[1].scatter(x_coords,y_coords,c=y_test,cmap='hot_r')
    ax[1].set_title('Truth')
    for i in range(len(landmarks)):
        ax[1].plot(landmarks[i][0],landmarks[i][1],marker='d',markersize=10,color='blue')
        

The aim of the following functions is to provide the implementations for the loss functions and the activation functions to be used in the generalized perceptron algorithm. In the following, the inputs y represent prediction(s), t the true value(s) and x the pre-activation value(s). These inputs can be scalars or vectors (both row or column vectors), that is these functions should work for inputs that are scalar or vectors.

In [10]:
def mse_loss(y, t):
    MSE=np.square(np.subtract(t,y)).mean()
    return MSE
    
def mse_loss_grad(y, t):
    y=np.array(y)
    t=np.array(t)
    return 2*np.subtract(t,y)/y.size
    
def binary_crossentropy_loss(y, t):
    t=np.array(t)
    y=np.array(y)
    eps=1e-10
    return -np.mean((t*np.log(y+eps)+(1-t)+np.log(1-y+eps)))

def binary_crossentropy_loss_grad(y, t):
    eps=1e-10
    return (t-y)/(t*(1-t)+eps)
    
def linear_activation(x):
    return x
    
def linear_activation_grad(x):
    if np.isscalar(x)==False:
        dim=np.shape(x)
        arr=np.ones(dim)
        return arr
    return 1
    
def logistic_activation(x):
    
    sigma=1/(1+np.exp(-x))
    return sigma
    
def logistic_activation_grad(x):
    s=logistic_activation(x)
    s_grad=s*(1-s)
    return s_grad
    
def relu_activation(x, alpha=.05):
    return np.maximum(alpha*x,x)
    
def relu_activation_grad(x, alpha=.05):
    
    return np.where(x>0,1,alpha)

The aim of the following functions is to train artificial neural network type of classifiers and regressors. The classification is always intended as binary classification. Here I have written my own implementation for the cases of the multi layer perceptron with backpropagation rule. This implementation is more general and can accept in input arbitrary activation and loss functions for the units in the hidden layer and the output unit. The multi layer perceptron is always assumed to have a single output unit and a single hidden layer.

Note here I have implemented a batch update, that is the model is not updated as it processes each instance, but rather the gradients are stored and averaged together whenever a number of instances equal to batch_size has been observed.

In the following perceptron_model = (W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad), that is the perceptron model is a 10-tuple containing the weight matrix W_ih, the bias vector b_ih, the weight vector W_ho and the bias scalar b_ho, the activation function for the units in the hidden layer hidden_activation_func, and its derivative hidden_activation_func_grad, the activation function for the unit in the output layer out_activation_func and its derivative out_activation_func_grad the loss function loss and the derivative of the loss function loss_grad. The suffix _ih stands for input to hidden, while _ho stands for hidden to output.

a) perceptron_model = init_multilayer_perceptron(in_dim, hidden_dim, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad, init_size=1e-3) that takes in input a desired integer input dimension in_dim, a desired integer hidden_dim to specify the number of units in the hidden layer, the desired activation functions for the units in the hidden layer and the output unit, and loss functions and a desired float size init_size. This function initializes the perceptron model with random values. The weight matrices, vectors and scalars are initialised with values in the range (-init_size, +init_size). The weight matrix W_ih has dimensionality equal to in_dim x hidden_dim, the bias vector b_ih is a column vector of size hidden_dim, the weight vector W_ho has size hidden_dim and b_ho is a scalar.

b) y or (y, a, h, h_ih)  = forward_multilayer_perceptron(x, perceptron_model, return_pre_activation=False) that takes in input a vector x and a model perceptron_model and returns the output prediction according to the perceptron algorithm. If the flag return_pre_activation is set to True then the output is a 4-tuple where the first element is the predicted output and the second element is the pre-activation value for the output unit, h is a vector containing the post-activations for the units in the hidden layer and h_ih is a vector containing the pre-activations for the units in the hidden layer. 

c) grad = compute_gradient_multilayer_perceptron(x, t, perceptron_model) that takes in input a vector x, a target value t, and a model perceptron_model and returns the gradients grad according to the perceptron algorithm. grad = (DW_ih, Db_ih, DW_ho, Db_ho) is a 4-tuple where the first element is the gradient of the weight matrix W_ih, the second element is the gradient of the vector b_ih, the third element is the gradient of the vector W_ho and the last element is the increment of the scalar b_ho. 

d) perceptron_model = update_multilayer_perceptron(grads, learning_rate, perceptron_model) that takes in input a list grads of gradients (each a 4-tuple), a learning rate float value of learning_rate and a model perceptron_model and returns an updated model according to the perceptron algorithm, that is, all the updates contained in grads are performed.

e) perceptron_model = fit_multilayer_perceptron(X_train, y_train, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad, learning_rate, hidden_dim, batch_size=10, max_n_iter=1000, verbose=False) that takes in input a data matrix X_train a target vector y_train, the desired activation functions for the units in the hidden layer and in the output layer, and loss function, all with their respective derivative functions, a learning rate float value of learning_rate, an integer to define the number of units in the hidden layer hidden_dim, an integer to define the batch size batch_size, that is how many instances are processed before updating the model, and
a maximum number of iterations of max_n_iter and returns a fit perceptron model according to the perceptron algorithm. If the flag verbose is set to True then every 100 iterations the loss is computed on the training data set and its value is printed to screen (this is useful to check that the loss is decreasing as training is progressing).

f) scores = score_multilayer_perceptron(X_test, perceptron_model) that takes in input a data matrix X_test, a fit perceptron model perceptron_model and returns a score vector scores containing real values such that larger positive values indicate a preference for the positive class and viceversa for negative values. 

g) preds = predict_multilayer_perceptron(X_test, perceptron_model) that takes in input a data matrix X_test, a fit perceptron model perceptron_model and returns a prediction vector preds containing values 1 when predicting the positive class and 0 when predicting the negative class. 

In [13]:
def init_multilayer_perceptron(in_dim, hidden_dim, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad, init_size=1e-3):
    
    b_ih=np.random.uniform(-init_size,init_size,(hidden_dim,1))
    W_ih=np.random.uniform(-init_size,init_size,(in_dim,hidden_dim))   
    b_ho=np.random.uniform(-init_size,init_size)
    W_ho=np.random.uniform(-init_size,init_size,hidden_dim)
    
    model=(W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad)
    return model
    
def forward_multilayer_perceptron(x, perceptron_model, return_pre_activation=False):
    W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad=perceptron_model
    b_ih=b_ih.T.flatten()
    h_ih=np.dot(x,W_ih)+b_ih
    h=hidden_activation_func(h_ih)
    
    a=np.dot(h,W_ho)+b_ho
    y=out_activation_func(a)
    
    if return_pre_activation==False:
        return y
    return (y, a, h, h_ih)
    
def compute_gradient_multilayer_perceptron(x, t, perceptron_model):
    W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad=perceptron_model
    y, a, h, h_ih=forward_multilayer_perceptron(x, perceptron_model, return_pre_activation=True)
    error_grad=loss_grad(y,t)
    derivative=out_activation_func_grad(y)
    delta=derivative*error_grad
    W_ho=delta*h
    b_ho=delta    
    grad_in=hidden_activation_func_grad(h)
    delta1=grad_in*(W_ho*delta)
    b_ih=delta
    b_ih=np.reshape(b_ih,(-1,1)) 
    delta=np.reshape(delta,(1,-1))
    x=np.reshape(x,(-1,1))
    W_ih=x@delta    
    return (W_ih, b_ih, W_ho, b_ho)
    
def update_multilayer_perceptron(grads, learning_rate, perceptron_model):
    DW_ih, Db_ih, DW_ho, Db_ho=grads
    
    W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad=perceptron_model 
    W_ih=W_ih+learning_rate*DW_ih
    b_ih=b_ih+learning_rate*Db_ih
    W_ho=W_ho+learning_rate*DW_ho
    b_ho=b_ho+learning_rate*Db_ho
    
    model=(W_ih, b_ih, W_ho, b_ho, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad)
    return model
    
def avg_grads(l,n):
    init=l[0]
    for i in range(1,n):
        init+=l[i]
    init=init/n
    return init


def fit_multilayer_perceptron(X_train, y_train, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad, learning_rate, hidden_dim, batch_size=10, max_n_iter=1000, verbose=False):
    in_dim=np.shape(X_train)[1]
    
    model=init_multilayer_perceptron(in_dim, hidden_dim, hidden_activation_func, hidden_activation_func_grad, out_activation_func, out_activation_func_grad, loss, loss_grad, init_size=1e-3)
    for j in range(max_n_iter):
        if verbose==True and j%100==0:
            preds=predict_multilayer_perceptron(X_train, model)
            l=loss(preds,y_train)
            print('Loss: ',l.mean())
        W_in=[]
        W_o=[]
        b_in=[]
        b_o=[]
        for i in range(np.shape(X_train)[0]):
            grads=compute_gradient_multilayer_perceptron(X_train[i], y_train[i], model)
            W_in.append(grads[0])
            b_in.append(grads[1])
            W_o.append(grads[2])
            b_o.append(grads[3])
            if (i+1)%batch_size==0:
                W_in=avg_grads(W_in,batch_size)
                b_in=avg_grads(b_in,batch_size)
                W_o=avg_grads(W_o,batch_size)
                b_o=avg_grads(b_o,batch_size)
                grads=(W_in,b_in,W_o,b_o)
                mod=update_multilayer_perceptron(grads, learning_rate, model)
                model=mod
                W_in=[]
                W_o=[]
                b_in=[]
                b_o=[]
    
    return model
    
def score_multilayer_perceptron(X_test, perceptron_model):
    scores=np.zeros(np.shape(X_test)[0],dtype=float)
    for i in range(np.shape(X_test)[0]):
        scores[i]=forward_multilayer_perceptron(X_test[i], perceptron_model,return_pre_activation=False)
        
    return scores
    
def predict_multilayer_perceptron(X_test, perceptron_model):
    preds=np.zeros(np.shape(X_test)[0])
    scores=score_multilayer_perceptron(X_test, perceptron_model)
   
    for i in range(len(scores)):
        
        if scores[i]>0:
            preds[i]=1
        else:
            preds[i]=0
    
    return preds

Checkpoint to assess whether the functions were implemented correctly

In [ ]:
# classification
data, y, pos_landmarks, neg_landmarks = make_dataset_classification(size=300, complexity=2, ndim=2, return_landmarks=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=.3)
X_test, y_test = make_2d_grid_dataset_classification(3000, pos_landmarks, neg_landmarks)

perceptron_model = fit_multilayer_perceptron(X_train, y_train, 
                                             relu_activation, relu_activation_grad, 
                                             logistic_activation, logistic_activation_grad, 
                                             binary_crossentropy_loss, binary_crossentropy_loss_grad, 
                                             learning_rate=1e-2, hidden_dim=4, batch_size=10, 
                                             max_n_iter=1000, verbose=False)
preds = predict_multilayer_perceptron(X_test, perceptron_model)
plot_2d_classification(X_test, y_test, preds, pos_landmarks, neg_landmarks)

# regression
data, y, landmarks = make_dataset_regression(size=300, complexity=2, ndim=2, return_landmarks=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=.3)
X_test, y_test = make_2d_grid_dataset_regression(3000, landmarks)

perceptron_model = fit_multilayer_perceptron(X_train, y_train, 
                                             relu_activation, relu_activation_grad, 
                                             linear_activation, linear_activation_grad, 
                                             mse_loss, mse_loss_grad, 
                                             learning_rate=1e-2, hidden_dim=4, batch_size=10, 
                                             max_n_iter=1000, verbose=False)
preds = score_multilayer_perceptron(X_test, perceptron_model)
plot_2d_regression(X_test, y_test, preds, landmarks)

<ipython-input-10-0804b03005a4>:32: RuntimeWarning: overflow encountered in exp
  sigma=1/(1+np.exp(-x))
<ipython-input-13-8181125eaae2>:33: RuntimeWarning: overflow encountered in multiply
  delta1=grad_in*(W_ho*delta)
<ipython-input-13-8181125eaae2>:17: RuntimeWarning: invalid value encountered in double_scalars
  a=np.dot(h,W_ho)+b_ho
